In [34]:
import guitarpro
import music21
import math
import os


def get_note(track, note):
    base_note = track.strings[note.string - 1]
    n1 = music21.note.Note(str(base_note))
    return n1.transpose(note.value)

# unique note number
def diatonicNoteName(note): 
    p1 = music21.pitch.Pitch(note.nameWithOctave)
    return p1.diatonicNoteNum
    
# midi note number
def midi(note): 
    p1 = music21.pitch.Pitch(note.nameWithOctave)
    return p1.midi
    
# midi note number
def frequency(note, multipler = 100): 
    p1 = music21.pitch.Pitch(note.nameWithOctave)
    return round(p1.frequency * multipler)      

def get_duration(beat):
    tuplet = beat.duration.tuplet
    tupletValue = tuplet.times / tuplet.enters
    duration = music21.duration.Duration(4 / beat.duration.value * tupletValue)
    if beat.duration.isDotted:
        duration.dots = 1
    return int(duration.quarterLength * 100000)

#https://www.midi.org/specifications/item/gm-level-1-sound-set - id for instrument
def is_bass_midi_instrument(instrument):
    if instrument in range(33, 40):
        return True
    return False

def get_bass_track(song):
    for track in song.tracks:
        if is_bass_midi_instrument(track.channel.instrument) or "bass" in track.name.lower():
            return track
        

def parse_song(file, output):
    song = guitarpro.parse(file)
    track = get_bass_track(song)
    if not track:
        return

    row["artist"] = song.artist
    row["album"]  = song.album
    row["title"]  = song.title
    #row["key"]    = song.key.name
    #row["tempo"]  = str(song.tempo)

    for measure in track.measures:
        for voice in measure.voices:
            for beat in voice.beats:
                #print(beat.__dict__)
               
                row["duration"] = str(get_duration(beat))
                if beat.status == guitarpro.BeatStatus.rest:
                    row["note"] = "R"
                    row["frequency"] = str(0)
                else:
                    row["2note"] = ""
                    row["2frequency"] = ""
                    row["no_of_notes"] = str(1)
                    row["chord"] = ""
                    if len(beat.notes) > 1:
                        row["no_of_notes"] = str(len(beat.notes))
                        c = music21.chord.Chord()
                        #odwroc
                        for note in beat.notes[::-1]:
                            musicnote = get_note(track, note)
                            c.add(musicnote.nameWithOctave.replace('-', ''))   
                        row["chord"] = c.commonName
                        
                        secondNote = beat.notes[-2]
                        musicnote = get_note(track, secondNote)
                        row["2note"] = musicnote.nameWithOctave.replace('-', '')
                        row["2frequency"] = str(frequency(musicnote))
                        
                    try:
                        rootNote = beat.notes[-1]
                        type = rootNote.type.name
                        row["type"] = type
                        if type == "dead":
                            row["note"] = "X"
                            row["frequency"] = ""
                        else:
                            musicnote = get_note(track, rootNote)
                            row["note"] = musicnote.nameWithOctave.replace('-', '')
                            row["frequency"] = str(frequency(musicnote))
                            row["hammer"] = str(rootNote.effect.hammer)
                            row["ghostNote"] = str(rootNote.effect.ghostNote)
                            row["muted"] = str(rootNote.effect.palmMute)
                            row["slide"] = str(len(rootNote.effect.slides))
                    except:
                        print('smth goes wrong')
                        
                #print(row.values())
                output.write(seperator.join(row.values()) + '\n')
        
row = {
    "artist": None,
    "album" : None,
    "title" : None,
    #"key"   : None,
    #"tempo" : None,
    "duration" : None,
    "no_of_notes" : str(1),
    "chord" : "",
    "type"  : None,
    "note"  : None,
    "frequency"  : None,
    "2note" : "",
    "2frequency" : "",
    "hammer" : None,
    "ghostNote" : None,
    "muted" : None,
    "slide" : None
} 

output = open('output/output.csv', 'w+')    
#write header
seperator = ', '
#print(seperator.join(row))
output.write(seperator.join(row) + '\n')

#parsing files
datadir = 'data/'
files = os.listdir(datadir)
for gpfile in files:  
    #if gpfile == "Test.gp4":
        parse_song(datadir + gpfile, output)
#close file                
output.close()

smth goes wrong


In [15]:
ch = music21.chord.Chord(["C4","A5"])
print(ch.commonName)

Major Thirteenth


In [94]:
#TODO - this value is not correct
def get_duration(val, dotted = False):
    #tuplet = beat.duration.tuplet
    #tupletValue = tuplet.times / tuplet.enters
    tupletValue = 1
    value = 4 / val
    if dotted:
        value = value * 1.5
    return value * tupletValue

def get_duration2(val, dotted=False):
    duration = music21.duration.Duration(4 / val * 0.3333333333333)
    if dotted:
        duration.dots = 1
    print(duration)
    

get_duration2(4)
get_duration2(16)
get_duration2(16, True)



<music21.duration.Duration 1/3>
<music21.duration.Duration 1/12>
<music21.duration.Duration 0.125>
